# Experiment template
*This is an experiment template, used to auto-generate notebooks for new experiments*

## Experiment description


##### Jupyter helpers:

In [1]:
%reload_ext autoreload
%autoreload

Define imports

In [16]:

from kmeans_poc.data import KMeansSampleDataLoader
from kmeans_poc.models import KMeansModel
from kmeans_poc.data_processing import DataProcessor
from kmeans_poc.experimentation import MlflowExperimentation
from kmeans_poc.evaluation import Evaluator, EvaluationMetrics


ImportError: cannot import name 'ExperimentRunner' from 'kmeans_poc.evaluation' (/Users/katyamust/anaconda3/envs/cont_review/lib/python3.7/site-packages/kmeans_poc-0.2.0-py3.7.egg/kmeans_poc/evaluation/__init__.py)

## Load data
*replace MyDataLoader with your DataLoader implementation*

In [10]:
data_loader = KMeansSampleDataLoader(dataset_name="generated",dataset_version=1.0 )
x = data_loader.get_dataset()

Define experimentation object, which will be used for logging the experiments parameters, metrics and artifacts
*Replace MlflowExperimentation if you use a different experimentation system*

In [11]:
experimentation = MlflowExperimentation()

Create preprocessor for handling data preprocessing, feature engineering etc.

In [12]:
class MyPreprocessor(DataProcessor):
    def apply(self, X):
        pass

    def apply_batch(self, X):
        pass

preprocessor = MyPreprocessor()


Create model/logic:

In [13]:
my_model = KMeansModel()

Define evaluation

In [14]:
class MyEvaluator(Evaluator):
    def evaluate(self, **kwargs) -> EvaluationMetrics:
        pass

evaluator = MyEvaluator()

Run experiment

In [20]:
from kmeans_poc import ExperimentRunner

experiment_runner = ExperimentRunner(
    model=my_model,
    X_train=x,
    y_train=None,
    X_test=x,
    y_test=None,
    data_loader=data_loader,
    log_experiment=True,
    experiment_logger=experimentation,
    evaluator=evaluator,
    experiment_name="Experiment",
)

results = experiment_runner.run()
print(results)


INFO: 'Experiment' does not exist. Creating a new experiment
Logging package in /Users/katyamust/anaconda3/envs/cont_review/lib/python3.7/site-packages/kmeans_poc-0.2.0-py3.7.egg/kmeans_poc


TypeError: evaluate() takes 1 positional argument but 3 were given